# sklearn-onnx: Convert your scikit-learn model into ONNX

sklearn-onnx enables you to convert models from sklearn-learn toolkits into ONNX.

- Introduction
- Tutorial
- API Summary
- Gallery of examples
- Convert a pipeline
- Converters with options
- Supported scikit-learn Models
- Issues, questions

You should look for existing issues or submit a new one. Sources are available on onnx/sklearn-onnx.

## ONNX version

The converter can convert a model for a specific version of ONNX. Every ONNX release is labelled with an opset number returned by function onnx_opset_version. This function returns the default value for parameter target opset (parameter target_opset) if it is not specified when converting the model. Every operator is versioned. The library chooses the most recent version below or equal to the targetted opset number for every operator. The ONNX model has one opset number for every operator domain, this value is the maximum opset number among all onnx nodes.

In [ ]:
from skl2onnx import __max_supported_opset__
print("Last supported opset:", __max_supported_opset__)

## Backend

sklearn-onnx converts models in ONNX format which can be then used to compute predictions with the backend of your choice. However, there exists a way to automatically check every converter with onnxruntime, onnxruntime-gpu. Every converter is tested with this backend.



## Quick start

ONNX Runtime provides an easy way to run machine learned models with high performance on CPU or GPU without dependencies on the training framework. Machine learning frameworks are usually optimized for batch training rather than for prediction, which is a more common scenario in applications, sites, and services. At a high level, you can:

1. Train a model using your favorite framework.

2. Convert or export the model into ONNX format. See ONNX Tutorials for more details.

3. Load and run the model using ONNX Runtime.

In this tutorial, we will briefly create a pipeline with scikit-learn, convert it into ONNX format and run the first predictions.

### Step 1: Train a model using your favorite framework

We’ll use the famous Iris datasets.

In [ ]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

In [ ]:
type(iris), iris.data.shape, iris.target.shape, iris.target

In [ ]:
iris.keys(), iris.frame, iris.target_names, iris.feature_names, iris.data_module

### Step 2: Convert or export the model into ONNX format
ONNX is a format to describe the machine learned model. It defines a set of commonly used operators to compose models. There are tools to convert other model formats into ONNX. Here we will use ONNXMLTools.

In [ ]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

Check the model size of the random forest model.

In [ ]:
!du -sh rf_iris.onnx

### Step 3: Load and run the model using ONNX Runtime
We will use ONNX Runtime to compute the predictions for this machine learning model.

In [ ]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [ ]:
input_name, label_name

In [ ]:
pred_onx

The module implements two functions: convert_sklearn and to_onnx. The first one was used in the previous examples, it requires two mandatory arguments:

- a scikit-learn model or a pipeline
- initial types

scikit-learn does not store information about the training dataset. It is not always possible to retrieve the number of features or their types. That’s why the function needs another argument called initial_types. In many cases, the training datasets is a numerical matrix X_train. 

Then it becomes `initial_type=[('X', FloatTensorType([None, X_train.shape[1]]))]`. X is the name of this unique input, the second term indicates the type and shape. The shape is `[None, X_train.shape[1]]`, the first dimension is the number of rows followed by the number of features.


The number of rows is undefined as the the number of requested predictions is unknown at the time the model is converted. The number of features is usually known. Let’s assume now the input is a string column followed by a matrix, then initial types would be:

In [ ]:
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx

initial_type = [('X', FloatTensorType([None, 4]))]
onx = to_onnx(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
X_train.shape, X_test.shape

In [ ]:
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})

type(pred_onx), pred_onx[0]

## Related converters

sklearn-onnx only converts models from scikit-learn. onnxmltools can be used to convert models for libsvm, lightgbm, xgboost. Other converters can be found on github/onnx, torch.onnx, ONNX-MXNet API, Microsoft.ML.Onnx…

# Train and deploy a scikit-learn pipeline


In [ ]:
# from pyquickhelper.helpgen.graphviz_helper import plot_graphviz
import numpy
from onnxruntime import InferenceSession
from sklearn.datasets import load_diabetes
from sklearn.ensemble import (
    GradientBoostingRegressor, RandomForestRegressor,
    VotingRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from skl2onnx import to_onnx
from mlprodict.onnxrt import OnnxInference


X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train classifiers
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=5)
reg2 = RandomForestRegressor(random_state=1, n_estimators=5)
reg3 = LinearRegression()

ereg = Pipeline(steps=[
    ('voting', VotingRegressor([('gb', reg1), ('rf', reg2), ('lr', reg3)])),
])
ereg.fit(X_train, y_train)

## Converts the model
The second argument gives a sample of the data used to train the model. It is used to infer the input type of the ONNX graph. It is converted into single float and ONNX runtimes may not fully support doubles.

In [ ]:
onx = to_onnx(ereg, X_train[:1].astype(numpy.float32),
              target_opset=12)

## Prediction with ONNX
The first example uses onnxruntime.

In [ ]:
sess = InferenceSession(onx.SerializeToString())
pred_ort = sess.run(None, {'X': X_test.astype(numpy.float32)})[0]

pred_skl = ereg.predict(X_test.astype(numpy.float32))

print("Onnx Runtime prediction:\n", pred_ort[:5])
print("Sklearn rediction:\n", pred_skl[:5])

## Comparison
Before deploying, we need to compare that both scikit-learn and ONNX return the same predictions.

In [ ]:
def diff(p1, p2):
    p1 = p1.ravel()
    p2 = p2.ravel()
    d = numpy.abs(p2 - p1)
    return d.max(), (d / numpy.abs(p1)).max()


print(diff(pred_skl, pred_ort))

It looks good. Biggest errors (absolute and relative) are within the margin error introduced by using floats instead of doubles. We can save the model into ONNX format and compute the same predictions in many platform using onnxruntime.